# Modules

In [ ]:
%matplotlib widget
import glob
import psfex
import pickle
import sncosmo
import numpy as np
import pandas as pd
import iminuit as im
import astropy as ap
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ztfquery
import ztfimg
import ztflc

from ztfimg import dao
from ztfimg import image
from ztflc import fitter
from ztflc import diffdata
from astropy.io import fits
from ztfquery import marshal
from astrobject import photometry
from ztflc import forcephotometry
from scipy.stats import multivariate_normal
from sncosmo.models import Source, Model, get_source
from sncosmo import get_bandpass, get_magsystem
from sncosmo.photdata import photometric_data

## Img loading

In [60]:
sciimg = "/home/nicolas/Work/Data/ztf/PSFEx_test/ztf_20190124095417_000403_zr_c01_o_q1_sciimg.fits"
mask = "/home/nicolas/Work/Data/ztf/PSFEx_test/ztf_20190124095417_000403_zr_c01_o_q1_mskimg.fits"
psFranck = "/home/nicolas/Work/Data/ztf/PSFEx_test/ztf_20190124095417_000403_zr_c01_o_q1_psfcat.fits"

hdul = fits.open(sciimg)
pdul = fits.open(psFranck)
header = hdul[0].header

psfcat = pd.DataFrame(pdul[1].data)

z = image.ScienceImage(sciimg, mask)
z.load_source_background()

row = z.header["NAXIS1"]
col = z.header["NAXIS2"]

In [29]:
psfchoice = ipw.Dropdown(options=['wrapper', 'ZOGY_default', 'ZOGY_FinalRun'],
                         description='PSF test',
                         value='ZOGY_default')
def choose_psf(psfdir):
    global psfimg
    psfilepath = "../../Data/ztf/PSFEx_test/config_ZOGY/" + psfdir + '/test.psf'
    psfobj = psfex.PSFEx(psfilepath)
    psfimg = psfobj.get_rec(row, col)

int_choose_psf = ipw.interactive(choose_psf, psfdir=psfchoice)
display(int_choose_psf)

interactive(children=(Dropdown(description='PSF test', index=1, options=('wrapper', 'ZOGY_default', 'ZOGY_Fina…

In [47]:
plt.close()
fig = plt.figure(figsize=[8, 3])

ax = fig.add_subplot(131)
ax.imshow(psfimg)
plt.title('PSF from ' + psfchoice.value)

ax = fig.add_subplot(132)
ax.imshow(d.basedata)
plt.title('PSF from Franck')

ax = fig.add_subplot(133)
ax.imshow(pdul[1].data)
#ax.imshow(d.basedata-psfimg)
plt.title('Difference')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Image data of dtype (numpy.record, [('sourceid', '<i4'), ('xpos', '<f4'), ('ypos', '<f4'), ('ra', '<f8'), ('dec', '<f8'), ('flux', '<f4'), ('sigflux', '<f4'), ('mag', '<f4'), ('sigmag', '<f4'), ('snr', '<f4'), ('chi', '<f4'), ('sharp', '<f4'), ('flags', '<i2')]) cannot be converted to float

In [20]:
FILE_OUT = sciimg[:-11] + 'sciunc.fits'

fake_sci = fits.HDUList(fits.PrimaryHDU(data=z.sourcebackground.rms(),
                                        header=header))
fake_sci.writeto(FILE_OUT, overwrite=True)

In [22]:
with open('../../Data/ztf/ZOGYexample/central_patch', 'rb') as f:
    central_patch = pickle.load(f)

## Norm values

In [6]:
fdata = ztfnewscipsf.rawdata
newsci_max = np.max(fdata)

In [24]:
central_norm = np.sum(np.ravel(central_patch))
central_normed = central_patch/central_norm
central_normed_max = np.max(central_normed)
central_scaled = central_normed*newsci_max/central_normed_max

In [28]:
plt.close()
fig = plt.figure(figsize=[10, 3])

ax = fig.add_subplot(131)
ax.imshow(central_scaled)
plt.title('Nora')

ax = fig.add_subplot(132)
ax.imshow(fdata)
plt.title('Frank')

ax = fig.add_subplot(133)
ax.imshow(central_scaled-fdata)
plt.title('Nora-Frank')

fig.savefig('../../Images/png/central_newscipsf_diff.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D Gaussian of Frank's psf patch

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from matplotlib.patches import Ellipse

# Generate fake data
np.random.seed(0)
y, x = np.mgrid[:25, :25]
z = fdata

# From Young-Lo
fwmh_x, fwmh_y = (7.869536e-01, 9.828351e-01)

fix_dict = {'x_mean': 12.5,
            'y_mean': 12.5}

# Fit the data using astropy.modeling
p_init = models.Gaussian2D(amplitude=0.05,
                           x_mean=12.5,
                           y_mean=12.5,
                           #fixed=fix_dict,
                           cov_matrix=np.asarray([[1, 0],
                                                  [0, 1]]))
fit_p = fitting.LevMarLSQFitter()
p = fit_p(p_init, x, y, z)

# Plot the data with the best-fit model
plt.close()
fig = plt.figure(figsize=[10, 3])

ax = plt.subplot(1, 3, 1)
ax.imshow(z, origin='lower', interpolation='nearest',
           vmin=0, vmax=newsci_max)
plt.title("Frank")

ax = plt.subplot(1, 3, 2)
ax.imshow(p(x, y), origin='lower', interpolation='nearest',
           vmin=0, vmax=newsci_max)
ellipse = Ellipse((p.parameters[1], p.parameters[2]),
                  width=p.parameters[3], height=p.parameters[4],
                  angle=p.parameters[5],
                  ec=None, fc=None, fill=False)
ax.add_patch(ellipse)
ellipse = Ellipse((p.parameters[1], p.parameters[2]),
                  width=2*p.parameters[3], height=2*p.parameters[4],
                  angle=p.parameters[5],
                  ec=None, fc=None, fill=False)
ax.add_patch(ellipse)
ellipse = Ellipse((p.parameters[1], p.parameters[2]),
                  width=3*p.parameters[3], height=3*p.parameters[4],
                  angle=p.parameters[5],
                  ec=None, fc=None, fill=False)
ax.add_patch(ellipse)

plt.title("BestFit2DGaussian")
ax = plt.subplot(1, 3, 3)
ax.imshow(z - p(x, y), origin='lower', interpolation='nearest',
           vmin=-0.0001, vmax=0.005)
plt.title("Residual")

fig.savefig('../../Images/png/frankpsf2dgauss.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
np.asarray([51, 0, 102])/255

array([0.2, 0. , 0.4])

In [60]:
p.parameters

array([ 0.04663146, 12.5       , 12.5       ,  1.66483445,  1.85841131,
       -0.84712916])

In [20]:
x, y = np.mgrid[-1:1:.01, -2:2:.01]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
rv = multivariate_normal([0.5, -0.2], [[2.0, 1], [1, 0.7]])
plt.close()
plt.contourf(x, y, multivariate_normal.pdf(pos, [0.5, -0.2], [[2.0, 1], [1, 0.7]]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
val = rv.pdf(pos)

In [29]:
x = np.linspace(0, 200, 201)
y = np.linspace(0, 200, 201)
x, y = np.meshgrid(x, y)

In [25]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.signal import argrelmax

import matplotlib.pyplot as plt
from matplotlib import cm

# 2D Gaussian model
def func(xy, x0, y0, sigma, H):

    x, y = xy

    A = 1 / (2 * sigma**2)
    I = H * np.exp(-A * ( (x - x0)**2 + (y - y0)**2))
    return I

# Generate 2D gaussian
def generate(x0, y0, sigma, H):

    x = np.arange(0, max(x0, y0) * 2 + sigma, 1)
    y = np.arange(0, max(x0, y0) * 2 + sigma, 1)
    xx, yy = np.meshgrid(x, y)

    I = func((xx, yy), x0=x0, y0=y0, sigma=sigma, H=H)

    return xx, yy, I

def fit(image, with_bounds):

    # Prepare fitting
    x = np.arange(0, image.shape[1], 1)
    y = np.arange(0, image.shape[0], 1)
    xx, yy = np.meshgrid(x, y)

    # Guess intial parameters
    x0 = int(image.shape[0]/2) # Middle of the image
    y0 = int(image.shape[1]/2) # Middle of the image
    sigma = max(*image.shape) * 0.1 # 10% of the image
    H = np.max(image) # Maximum value of the image
    initial_guess = [x0, y0, sigma, H]

    # Constraints of the parameters
    if with_bounds:
        lower = [0, 0, 0, 0]
        upper = [image.shape[0], image.shape[1], max(*image.shape), image.max() * 2]
        bounds = [lower, upper]
    else:
        bounds = [-np.inf, np.inf]

    pred_params, uncert_cov = curve_fit(func, (xx.ravel(), yy.ravel()), image.ravel(),
                                        p0=initial_guess, bounds=bounds)

    # Get residual
    predictions = func((xx, yy), *pred_params)
    rms = np.sqrt(np.mean((image.ravel() - predictions.ravel())**2))

    print("Predicted params : ", pred_params)
    print("Residual : ", rms)

    return pred_params

def plot(image, params):

    fig, ax = plt.subplots()
    ax.imshow(image, cmap='viridis', interpolation='nearest', origin='lower')

    ax.scatter(params[0], params[1], s=100, c="red", marker="x")

    circle = Circle((params[0], params[1]), params[2], facecolor='none',
            edgecolor="red", linewidth=1, alpha=0.8)
    ax.add_patch(circle)
    circle = Circle((params[0], params[1]), 2*params[2], facecolor='none',
        edgecolor="green", linewidth=1, alpha=0.8)
    ax.add_patch(circle)
    circle = Circle((params[0], params[1]), 3*params[2], facecolor='none',
        edgecolor="black", linewidth=1, alpha=0.8)
    ax.add_patch(circle)

In [26]:
params = fit(fdata, with_bounds=False)
plot(fdata, params)

Predicted params :  [12.00242999 12.04015469  1.64680838  0.05176076]
Residual :  0.0005797823351933234


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …